# Anonymous Feature Correlation Notebook

This notebook correlates the anonymous features from `train.csv` with public, open-source data from **Yahoo Finance** and **FRED (Federal Reserve Economic Data)**.

**Objective:** To de-anonymize the features by finding strong correlations with real-world indicators.

**Logic:**
1.  **Install** required libraries.
2.  **Load Public Data:** Download 10+ years of daily data from Yahoo (`yfinance`) and FRED (`fredapi`).
3.  **Load Anonymous Data:** Load the `train.csv` file.
4.  **Map Dates:** Replicate the logic from the original notebook. We assume the `N` unique `date_id`s in `train.csv` map sequentially to the *last N* trading days of the S&P 500 (`^GSPC`) data.
5.  **Merge Data:** Join all dataframes on the mapped `Date`.
6.  **Calculate & Show Correlations:** Find the strongest (positive and negative) correlations between the anonymous and public features.

In [1]:
%pip install polars pandas yfinance fredapi --quiet

DEPRECATION: Loading egg at /home/paul/anaconda3/envs/ml_311/lib/python3.11/site-packages/ete3-3.1.3-py3.10.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [1]:
import polars as pl
import pandas as pd
import yfinance as yf
from fredapi import Fred
import io

print("Libraries imported.")

Libraries imported.


## Part 1: Load Open-Source Public Data

First, we define our date range and the public tickers/series we want to check.

In [2]:
# --- Parameters ---
START_DATE = "2014-01-01"
END_DATE = "2024-12-31"

# 1. Yahoo Finance Tickers
# (Market, Price, Volatility)
YFINANCE_TICKERS = {
    "^GSPC": "SPY",  # S&P 500 Index (Our base for date mapping)
    "^VIX": "VIX",   # Volatility Index
    #"^IXIC": "NASDAQ", # NASDAQ Composite
    "GLD": "GOLD",   # Gold ETF
    "USO": "OIL"     # Oil Fund ETF
}

# 2. FRED Series IDs
# (Macro-Economic, Interest Rate)
FRED_SERIES = {
    "DGS10": "T10Y",    # 10-Year Treasury Yield
    "DGS2": "T2Y",     # 2-Year Treasury Yield
    "T10Y2Y": "YIELD_CURVE", # 10-2 Year Spread
    "DFF": "FED_FUNDS", # Federal Funds Rate
    "DTWEXBGS": "DOLLAR_IDX" # US Dollar Index
}

# 3. Path to your anonymous training data
# !!! UPDATE THIS PATH !!!
TRAIN_DATA_PATH = "./kaggle/train.csv"
SPY_DATE_PATH = "./kaggle/spy-historical.csv"  # Public SPY dates data

In [56]:
# --- 2. Load Public Data ---
import yfinance as yf
from fredapi import Fred
import polars as pl
from datetime import datetime
import pandas as pd
import dotenv

# *** START OF UPDATE ***

# --- Configuration ---
START_DATE = "2010-01-01"
END_DATE = datetime.now().strftime('%Y-%m-%d')
FRED_API_KEY = dotenv.get_key(dotenv.find_dotenv(), "FRED_API_KEY")

# Define tickers and FRED series IDs by category
YFINANCE_TICKERS = {
    # M* - Market Dynamics/Technical
    'M_GSPC': '^GSPC',        # S&P 500 Index
    'M_IXIC': '^IXIC',        # NASDAQ Composite
    'M_DJI': '^DJI',          # Dow Jones Industrial Average
    'M_RUT': '^RUT',          # Russell 2000

    # V* - Volatility
    'V_VIX': '^VIX',          # CBOE Volatility Index
    #'V_VXN': '^VXN',          # CBOE NASDAQ 100 Volatility Index
    'V_VIX3M': '^VIX3M',      # CBOE 3 Month Volatility Index

    # P* - Price/Valuation
    'P_GC=F': 'GC=F',         # Gold Futures
    'P_CL=F': 'CL=F',         # Crude Oil Futures
    'P_SI=F': 'SI=F',         # Silver Futures
    'P_HG=F': 'HG=F',         # Copper Futures

    # I* - Interest Rates (from Yahoo)
    'I_FVX': '^FVX',          # Treasury Yield 5 Years
    'I_TNX': '^TNX',          # CBOE Interest Rate 10 Year T Note
    'I_TYX': '^TYX',          # Treasury Yield 30 Years
    
    # S* - Sentiment/Alternative
    'S_BTC-USD': 'BTC-USD',   # Bitcoin
}

FRED_SERIES = {
    # E* - Macro Economic
    'E_GDP': 'GDP',                       # Gross Domestic Product
    'E_UNRATE': 'UNRATE',                 # Unemployment Rate
    'E_CPIAUCSL': 'CPIAUCSL',             # Consumer Price Index
    'E_PPIACO': 'PPIACO',                 # Producer Price Index
    'E_INDPRO': 'INDPRO',                 # Industrial Production Index
    'E_PAYEMS': 'PAYEMS',                 # Non-Farm Payrolls
    'E_ICSA': 'ICSA',                     # Initial Claims

    # I* - Interest Rates
    'I_DFF': 'DFF',                       # Federal Funds Effective Rate
    'I_DTB3': 'DTB3',                     # 3-Month Treasury Bill
    'I_DGS2': 'DGS2',                     # 2-Year Treasury Yield
    'I_DGS10': 'DGS10',                   # 10-Year Treasury Yield
    'I_T10Y2Y': 'T10Y2Y',                 # 10-Year vs 2-Year Treasury Spread

    # V* - Volatility
    # 'V_VIXCLS': 'VIXCLS',                 # VIX (from FRED)

    # S* - Sentiment
    'S_UMCSENT': 'UMCSENT',               # University of Michigan Consumer Sentiment
    'S_BAMLH0A0HYM2': 'BAMLH0A0HYM2',    # BofA US High Yield Index Option-Adjusted Spread
    'S_EMVMACROBUS': 'EMVMACROBUS',      #Equity Market Volatility Tracker: Macroeconomic News and Outlook: Business Investment And Sentiment
}



# --- Data Fetching Functions ---
def fetch_yfinance_data(tickers, start, end):
    """Fetches and processes data from Yahoo Finance."""
    print(f"Fetching {len(tickers)} tickers from Yahoo Finance...")
    df = yf.download(list(tickers.values()), start=start, end=end, progress=False)
    
    # Use only 'Adj Close' and rename columns to custom names
    df = df['Close']
    df = df.rename(columns={v: k for k, v in tickers.items()})
    
    df = df.reset_index()
    return pl.from_pandas(df)

def fetch_fred_data(series_ids, api_key, start, end):
    """Fetches and processes data from FRED."""
    print(f"Fetching {len(series_ids)} series from FRED...")
    try:
        fred = Fred(api_key=api_key)
    except ValueError as e:
        print(f"Error initializing Fred API: {e}")
        print("Please make sure you have set a valid FRED_API_KEY.")
        return pl.DataFrame()

    # Fetch all series
    df_list = []
    for code, name in series_ids.items():
        try:
            s = fred.get_series(name, start_date=start, end_date=end)
            s.name = code
            df_list.append(s)
        except Exception as e:
            print(f"Could not fetch series {name} ({code}): {e}")
            
    if not df_list:
        print("No data fetched from FRED.")
        return pl.DataFrame()
    
    # Combine into a single DataFrame
    df = pd.concat(df_list, axis=1).reset_index()
    df = df.rename(columns={'index': 'Date'})
    return pl.from_pandas(df)

# --- Fetch and Process Data ---
yf_data = fetch_yfinance_data(YFINANCE_TICKERS, START_DATE, END_DATE)
fred_data = fetch_fred_data(FRED_SERIES, FRED_API_KEY, START_DATE, END_DATE)


Fetching 14 tickers from Yahoo Finance...
Fetching 15 series from FRED...


/tmp/ipykernel_77516/3695140231.py:76: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(list(tickers.values()), start=start, end=end, progress=False)


In [63]:
# Define momentum periods (in days)
MOMENTUM_PERIODS = [21] # 1 week, 1 month, 6 months
MOMENTUM_PERIODS = []
# Ensure Date columns are of the same type
yf_data = yf_data.with_columns(pl.col("Date").cast(pl.Date))
if not fred_data.is_empty():
    fred_data = fred_data.with_columns(pl.col("Date").cast(pl.Date))

    # Join the two public datasets
    public_data = yf_data.join(fred_data, on='Date', how='left')
else:
    public_data = yf_data

# Sort and forward-fill missing values (common for economic data)
public_data = public_data.sort("Date").fill_null(strategy='forward')

# --- Calculate Momentum Features ---
mom_features = []
for col in public_data.columns:
    if col != 'Date': # Don't calculate momentum on the date column
        for period in MOMENTUM_PERIODS:
            # Calculate rolling average (SMA)
            sma_col_name = f"SMA_{period}D_{col}"
            public_data = public_data.with_columns(
                pl.col(col).rolling_mean(window_size=period).alias(sma_col_name)
            )
            
            # # Calculate Rate of Change (ROC)
            # roc_col_name = f"MOM_ROC_{period}D_{col}"
            # public_data = public_data.with_columns(
            #     ((pl.col(col) / pl.col(col).shift(period)) - 1).alias(roc_col_name)
            # )
            

# Drop rows with nulls created by momentum calculations
# public_data = public_data.drop_nulls()

print("\n--- Public Data Shape ---")
print(public_data.shape)
print("\n--- Public Data Head ---")
print(public_data.head())

# *** END OF UPDATE ***


--- Public Data Shape ---
(5234, 30)

--- Public Data Head ---
shape: (5, 30)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ Date      ┆ S_BTC-USD ┆ P_CL=F    ┆ P_GC=F    ┆ … ┆ I_T10Y2Y ┆ S_UMCSENT ┆ S_BAMLH0A ┆ S_EMVMACR │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---      ┆ ---       ┆ 0HYM2     ┆ OBUS      │
│ date      ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64      ┆ f64       ┆ ---       ┆ ---       │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 2010-01-0 ┆ null      ┆ 81.510002 ┆ 1117.6999 ┆ … ┆ 2.76     ┆ null      ┆ 6.34      ┆ null      │
│ 4         ┆           ┆           ┆ 51        ┆   ┆          ┆           ┆           ┆           │
│ 2010-01-0 ┆ null      ┆ 81.769997 ┆ 1118.0999 ┆ … ┆ 2.76     ┆ null      ┆ 6.3       ┆ null      │
│ 5         

In [64]:
# def load_yahoo_data(tickers, start, end):
#     """Downloads data from yfinance and formats it as a Polars DataFrame."""
#     print(f"Downloading {len(tickers)} tickers from Yahoo Finance...")
#     df = yf.download(list(tickers.keys()), start=start, end=end)
    
#     # Convert to Polars, reset index to get 'Date' column
#     df_pl = pl.from_pandas(df.stack().reset_index())
#     df_pl = df_pl.with_columns(pl.col("Date").cast(pl.Date))
    
#     # Pivot to wide format
#     df_wide = df_pl.pivot(index="Date", columns="Ticker", values="Close")
    
#     # Rename columns to be friendly
#     for ticker, name in tickers.items():
#         df_wide = df_wide.rename({ticker: f"{name}_Close"})
        
#     return df_wide.sort("Date")

# df_yahoo = load_yahoo_data(YFINANCE_TICKERS, START_DATE, END_DATE)
# print("Yahoo Finance Data:")
# print(df_yahoo.tail())

In [65]:
# import dotenv
# def load_fred_data(series_map):
#     """Downloads data from FRED and formats it as a Polars DataFrame."""
#     print(f"\nDownloading {len(series_map)} series from FRED...")
#     print("Note: This requires a free API key.")
    
#     # !!! UPDATE THIS KEY !!!
#     # Get one here: https://fred.stlouisfed.org/docs/api/api_key.html
#     API_KEY = dotenv.get_key(".env", "FRED_API_KEY")
    
#     try:
#         fred = Fred(api_key=API_KEY)
        
#         # Download all series
#         df_list = []
#         for series_id, name in series_map.items():
#             s = fred.get_series(series_id, start_date=START_DATE, end_date=END_DATE)
#             s.name = name
#             df_list.append(s)
            
#         # Combine and format
#         df_pd = pd.concat(df_list, axis=1)
#         df_pl = pl.from_pandas(df_pd.reset_index())
#         df_pl = df_pl.rename({"index": "Date"})
#         df_pl = df_pl.with_columns(pl.col("Date").cast(pl.Date))
#         return df_pl.sort("Date")
        
#     except Exception as e:
#         print(f"Error connecting to FRED. Did you set your API key? {e}")
#         return None

# df_fred = load_fred_data(FRED_SERIES)
# if df_fred is not None:
#     print("FRED Data (raw):")
#     print(df_fred)

## Part 2: Load and Map Anonymous Data

Here, we replicate the **exact date-mapping logic** from your original notebook. We use the `SPY_Close` data (downloaded as `^GSPC`) as the ground truth for trading days, just as your notebook used `spy-historical.csv`.

In [66]:

# Run the mapping
df_spy = pl.read_csv(SPY_DATE_PATH)
df_spy_date = df_spy.with_columns(pl.col("Date").str.to_date().alias("Date"))


"""Loads train.csv and maps date_id to actual dates."""

df_train = pl.read_csv(TRAIN_DATA_PATH)
df_train = df_train.slice(1000)
#print(df_train.glimpse())
anonymous_features = [col for col in df_train.columns if any(col.startswith(p) for p in ['M', 'E', 'I', 'P', 'V', 'S', 'D', 'MOM'])]
string_features_to_convert = []

for col_name in anonymous_features:
    if df_train[col_name].dtype == pl.String:
        string_features_to_convert.append(col_name)

if string_features_to_convert:
    print(f"Found {len(string_features_to_convert)} string columns to convert: {string_features_to_convert}")
    
    # 4. Convert them, turning bad values (like "N/A") into nulls
    df_train = df_train.with_columns(
        pl.col(string_features_to_convert).cast(pl.Float64, strict=False)
    )

# 1. Get unique, sorted date_ids from training data
unique_date_ids = df_train["date_id"].unique().sort()
n_dates_train = len(unique_date_ids)
print(f"Found {n_dates_train} unique date_ids in train.csv.")

# 2. Get the *last N* trading days from our public SPY data
# (This is the CRITICAL ASSUMPTION from your notebook)
spy_ground_truth_dates = df_spy_date.sort("date_id").tail(n_dates_train)["Date"]

if len(spy_ground_truth_dates) != n_dates_train:
    print("Error: Mismatch in date counts. Cannot perform mapping.")

# 3. Create the mapping dataframe
date_map = pl.DataFrame({
    "date_id": unique_date_ids,
    "Date": spy_ground_truth_dates
})

# print("Date mapping created. Example:")
# print(date_map.head(3))
# print("...")
# print(date_map.tail(3))

# 4. Join the map back to the training data
df_train_with_dates = df_train.join(date_map, on="date_id", how="left")
    


Found 85 string columns to convert: ['E1', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E2', 'E20', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'M1', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P1', 'P10', 'P11', 'P12', 'P13', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'V1', 'V10', 'V11', 'V12', 'V13', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']
Found 7990 unique date_ids in train.csv.


## Part 3: Merge All Data Sources

Now we combine everything into one master dataframe. We'll use the `df_train_with_dates` as our base, then join the Yahoo and FRED data to it.

We must **forward-fill** the FRED data, as it's reported on all days (including weekends) or only when it changes (like `FED_FUNDS`). Forward-filling applies the last known value to the trading days.

In [67]:

# Join all dataframes
df_merged = df_train_with_dates.join(public_data, on="Date", how="left")

# Forward-fill nulls in public data (especially for FRED data)
public_feature_cols = list(YFINANCE_TICKERS.keys()) + list(FRED_SERIES.keys())
#public_feature_cols = [f"{col}_Close" if col in YFINANCE_TICKERS.values() else col for col in public_feature_cols]
public_feature_cols = public_data.columns[1:]  # Exclude 'Date' column

df_merged = df_merged.with_columns(
    pl.col(public_feature_cols).forward_fill()
)

# Drop any remaining nulls (e.g., at the very start of the history)
df_merged = df_merged.drop_nulls().drop_nans()

print("Master dataframe created. Shape:", df_merged.shape)
print(df_merged.tail())
df_numeric_features = df_merged.select([pl.selectors.numeric()])
print(f"\nNumeric features dataframe shape: {df_numeric_features.shape}")

Master dataframe created. Shape: (2021, 128)
shape: (5, 128)
┌─────────┬─────┬─────┬─────┬───┬──────────┬───────────┬────────────────┬───────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆ I_T10Y2Y ┆ S_UMCSENT ┆ S_BAMLH0A0HYM2 ┆ S_EMVMACROBUS │
│ ---     ┆ --- ┆ --- ┆ --- ┆   ┆ ---      ┆ ---       ┆ ---            ┆ ---           │
│ i64     ┆ i64 ┆ i64 ┆ i64 ┆   ┆ f64      ┆ f64       ┆ f64            ┆ f64           │
╞═════════╪═════╪═════╪═════╪═══╪══════════╪═══════════╪════════════════╪═══════════════╡
│ 8985    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ 0.59     ┆ 58.2      ┆ 2.83           ┆ 0.0           │
│ 8986    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ 0.56     ┆ 58.2      ┆ 2.84           ┆ 0.0           │
│ 8987    ┆ 0   ┆ 0   ┆ 1   ┆ … ┆ 0.54     ┆ 58.2      ┆ 2.87           ┆ 0.0           │
│ 8988    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ 0.5      ┆ 58.2      ┆ 2.84           ┆ 0.0           │
│ 8989    ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ 0.49     ┆ 58.2      ┆ 2.78           ┆ 0.0           │
└─────────┴─────┴─────┴─────┴───┴──────

## Part 4: Calculate and Display Correlations

Finally, we calculate the correlation matrix. To make it easy to read, we won't print the whole matrix. Instead, we'll:

1.  Isolate the correlations between *anonymous* and *public* features.
2.  "Melt" the matrix into a long list: `(anonymous_feature, public_feature, correlation)`.
3.  Sort by the **absolute correlation value** to find the strongest relationships (both positive and negative).

In [69]:

# Define our two feature groups
public_features = public_feature_cols
#anonymous_features = [col for col in df_numeric_features.columns if col not in public_features and any(col.startswith(p) for p in ['M', 'E', 'I', 'P', 'V', 'S', 'D', 'MOM'])]

all_features = anonymous_features + public_features

print(f"\nAnonymous features ({len(anonymous_features)}): {anonymous_features}")
print(f"Public features ({len(public_features)}): {public_features}")

print(f"Correlating {len(anonymous_features)} anonymous features with {len(public_features)} public features...")

# Calculate the full correlation matrix for our subset of columns
corr_matrix = df_merged.select(anonymous_features + public_features).corr()

# *** START OF FIX ***

# Manually add the feature names as a new column
# This is the column we will filter on.
corr_matrix_with_labels = corr_matrix.with_columns(
    pl.Series("anonymous_feature", all_features)
)

# Filter the matrix to only show (Anonymous Rows x Public Columns)
corr_subset = corr_matrix_with_labels.filter(
    pl.col("anonymous_feature").is_in(anonymous_features)
).select(
    ["anonymous_feature"] + public_features  # Use the new column name
)

# Melt the matrix to a long format for easy sorting
corr_long = corr_subset.unpivot(
    index="anonymous_feature",  # Use the new column name
    variable_name="public_feature", 
    value_name="correlation"
)

# *** END OF FIX ***

# Sort by absolute correlation to find strongest links
corr_sorted = corr_long.with_columns(
    pl.col("correlation").abs().alias("abs_correlation")
).sort("abs_correlation", descending=True)

Notable_features = ['V13', 'M1', 'S5', 'M4']
for x in Notable_features:
    print(corr_sorted.filter(pl.col("anonymous_feature") == x).head(10))

print("\n--- Top 100 Most Correlated Feature Pairs --- (Positive or Negative)")
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=1000, tbl_rows=1000):
    print(corr_sorted.head(100))

print("\n--- Top 10 Positive Correlations ---")
print(corr_sorted.sort("correlation", descending=True).head(10))

print("\n--- Top 10 Negative Correlations ---")
print(corr_sorted.sort("correlation", descending=False).head(10))




Anonymous features (94): ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E2', 'E20', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'M1', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P1', 'P10', 'P11', 'P12', 'P13', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'S1', 'S10', 'S11', 'S12', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'V1', 'V10', 'V11', 'V12', 'V13', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']
Public features (29): ['S_BTC-USD', 'P_CL=F', 'P_GC=F', 'P_HG=F', 'P_SI=F', 'M_DJI', 'I_FVX', 'M_GSPC', 'M_IXIC', 'M_RUT', 'I_TNX', 'I_TYX', 'V_VIX', 'V_VIX3M', 'E_GDP', 'E_UNRATE', 'E_CPIAUCSL', 'E_PPIACO', 'E_INDPRO', 'E_PAYEMS', 'E_ICSA', 'I_DFF', 'I_DTB3', 'I_DGS2', 'I_DGS10', 'I_T10Y2Y', 'S_UMCSENT', 'S_BAMLH0A0HYM2', 'S_EMVMACROBUS']
Correlating 94 anonymous features wi